In [1]:
import requests
import urllib.request
import time
from bs4 import BeautifulSoup
from selenium import webdriver
import re

In [2]:
#routard

def scrap_routard(continent, country, city):
    """continent, country, city in FRENCH"""
    response = requests.get(f'https://www.routard.com/fr/guide/{continent}/{country}/{city}')
    routard_soup = BeautifulSoup(response.text, 'html.parser')
    links = routard_soup.find_all('a', attrs={'href': re.compile(f'guide/{continent}/{country}/{city}')})
    for link in links:
        print(link.get('href').split("/")[-1])
    # scraper les liens de chaque truc pour récup la description rapide ?

scrap_routard('europe', 'grece', 'athenes')

acropole
agora
erechtheion
jardin-national
mont-lycabette
musee-archeologique-national
musee-de-l-acropole
parlement-place-syndagma
parthenon
quartier-de-plaka


In [3]:
#guide michelin
region = 'attica'
city = 'athens'

In [4]:
COSTS = ['Affordable', 'Mid-Range', 'Premium', 'Luxury']
def scrap_michelin(region, city=None):
    data = {}
    if city is None:
        url = f"https://guide.michelin.com/fr/fr/{region}-region/restaurants?sort=distance"
    else:
        url = f"https://guide.michelin.com/fr/fr/{region}/{city}/restaurants?sort=distance"
    response = requests.get(url)
    michelin_soup = BeautifulSoup(response.text, 'html.parser')
    restaurants = michelin_soup.find_all('div', {'class': "card__menu-content"})
    if len(restaurants)==0:
        return None
    for resto in restaurants:
        resto_a = resto.find('a')
        name = resto_a.text.strip()
        link = "https://guide.michelin.com" + resto_a.get('href')
        nb_stars = 0
        bib_gourmand = False #bib-gourmand: nos meilleurs rapports qualité-prix
        gastronomie_durable = False
        gastronomie_durable = False
        for span in resto.find_all('span', {'class':'distinction-icon'}):
            for distinction in span.find_all('img'):
                src = distinction.get('src')
                if "star" in src:
                    nb_stars += 1
                elif "bib-gourmand" in src:
                    bib_gourmand = True
                elif "gastronomie-durable" in src:
                    gastronomie_durable = True
    
        resto_div = resto.find_all('div', {'class': "card__menu-footer--score"})
        tmp = re.sub(r'\s+', ' ', resto_div[1].text).strip() # to get "<$/€> · <cuisine>"
        price, cuisine = tmp.split(" · ")
        cost = COSTS[len(price)-1]
        address, latitude, longitude = scrap_resto_link(link)
        data[name] = {
            "link": link,
            "stars": nb_stars,
            "bib-gourmand": bib_gourmand,
            "cuisine": cuisine,
            "cost": cost,
            "address": address,
            "latitude":latitude,
            "longitude": longitude,
            "gastonomie_durable": gastronomie_durable
        }
    return data

def scrap_resto_link(url):
    resto_req = requests.get(url)
    resto_soup = BeautifulSoup(resto_req.text, 'html.parser')
    address = ""
    for div in resto_soup.find_all('div', {'class':'data-sheet__block--text'}):
        text = div.text.strip()
        if not '\n' in text:
            address = text
            break
    latitude, longitude = None, None
    for iframe in resto_soup.find_all('iframe'):
        i_src = iframe.get('src')
        if i_src is None:
            continue
        if "maps" in i_src:
            search = re.search(r'q=([-+]?[0-9]*\.?[0-9]+),([-+]?[0-9]*\.?[0-9]+)', i_src)
            if search:
                latitude = float(search.group(1))
                longitude = float(search.group(2))
    return address, latitude, longitude

# print(scrap_michelin('attica', 'athens'))

In [5]:
# https://guide.michelin.com/fr/fr/attica/athens/restaurants/page/2?sort=distance
# faire les pages suivantes ?

In [6]:
import json # for print
# https://guide.michelin.com/fr/fr/osaka-region/restaurants
region = "osaka"
city = "osaka"
data = scrap_michelin(region, city)
if data is None:
    data = scrap_michelin(region)
print(json.dumps(data, indent=4))

{
    "YUNiCO": {
        "link": "https://guide.michelin.com/fr/fr/osaka-region/osaka/restaurant/yunico-1194638",
        "stars": 1,
        "bib-gourmand": false,
        "cuisine": "Cuisine italienne",
        "cost": "Premium",
        "address": "4F, 1-5-17 Sonezakishinchi, Kita-ku, Osaka, 530-0002, Japon",
        "latitude": 34.69807,
        "longitude": 135.496874,
        "gastonomie_durable": false
    },
    "TAVERNETTA da KITAYAMA": {
        "link": "https://guide.michelin.com/fr/fr/osaka-region/osaka/restaurant/tavernetta-da-kitayama",
        "stars": 0,
        "bib-gourmand": false,
        "cuisine": "Cuisine italienne",
        "cost": "Mid-Range",
        "address": "1-6 Ishigatsujicho, Tennoji-ku, Osaka, 543-0031, Japon",
        "latitude": 34.664805,
        "longitude": 135.523061,
        "gastonomie_durable": false
    },
    "Chukasoba Mugen": {
        "link": "https://guide.michelin.com/fr/fr/osaka-region/osaka/restaurant/chukasoba-mugen",
        "stars"

## Suppr stop words 13 langues

In [ ]:
!pip install langdetect

In [ ]:
from langdetect import detect
from nltk.corpus import stopwords
nltk.download('stopwords')

language_dict = {
  'cs': 'czech',
  'da': 'danish',
  'nl': 'dutch',
  'en':'english',
  'et': 'estonian',
  'fi': 'finnish',
  'fr': 'french',
  'de': 'german',
  'el': 'greek',
  'it': 'italian',
  'no': 'norwegian',
  'pl': 'polish',
  'pt': 'portuguese',
  'ru': 'russian',
  'sl': 'slovene',
  'es': 'spanish',
  'sv': 'swedish',
  'tr': 'turkish'
}


def tokenize_without_stopwords(text):
  try:
    language = language_dict[detect(text)]
    stop_words = set(stopwords.words(language))
    filtered_text = " ".join([word for word in text.split() if word not in stop_words])
    return word_tokenize(filtered_text, language=language)

  except Exception as e:
    if len(str(e)) == 4:
      print("[WARNING] language not supported:", str(e))
    else:
      print("error", e)
    return word_tokenize(text)